In [3]:
import os

import pytorch_lightning as pl
# your favorite machine learning tracking tool
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader

from torchmetrics import Accuracy

from torchvision import transforms
from torchvision.datasets import StanfordCars
from torchvision.datasets.utils import download_url
import torchvision.models as models

import wandb

In [4]:
class StanfordCarsDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # Augmentation policy for training set
        self.augmentation = transforms.Compose([
              transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
              transforms.RandomGrayscale(p=0.5),
              transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
              transforms.RandomPosterize(bits=2, p=0.5),
            
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        # Preprocessing steps applied to validation and test set.
        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
        self.num_classes = 196

    def prepare_data(self):
        StanfordCars(root=self.data_dir, download=True, split="train")
        StanfordCars(root=self.data_dir, download=True, split="test")

    def setup(self, stage=None):
        # build dataset
        self.train = StanfordCars(root=self.data_dir, split="train", transform=self.augmentation)
        # split dataset
        # self.train, self.val = random_split(dataset, [0.6, 0.4])

        self.val, self.test = random_split(StanfordCars(root=self.data_dir, split="test", transform=self.transform), [0.5, 0.5])
        
        # self.test = random_split(self.test, [len(self.test)])[0]

        # self.train.dataset.transform = self.augmentation
        # self.val.dataset.transform = self.transform
        # self.test.dataset.transform = self.transform
        
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=2)

In [5]:
from torch.nn.modules.linear import Linear
class LitModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4, transfer=False):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes
        # transfer learning if pretrained=True
        self.feature_extractor = models.efficientnet_b1(pretrained=transfer)

        if transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = True
                
        self.feature_extractor.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes)
        
        # n_sizes = self._get_conv_output(input_shape)

        # self.classifier = nn.Linear(n_sizes, num_classes)
        
        # for param in self.classifier.parameters():
        #         param.requires_grad = True

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=196)
  
    # returns the size of the output tensor going into the Linear layer from the conv block.
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
    # returns the feature tensor from the conv block
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        return x

    def IC(input):
      eps = 1.1e-5
      x = nn.BatchNorm2d(num_features=input.shape[1], eps=eps)(input)
      x = x * torch.rsqrt(nn.BatchNorm2d(num_features=input.shape[1], eps=eps).running_var + eps)
      x = nn.Dropout(0.01)(x)
      return x

    # will be used during inference
    def forward(self, x):
       x = self._forward_features(x)
       # x = x.view(x.size(0), -1)
       # x = self.classifier(x)
       
       return x
    
    def training_step(self, batch):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        acc = self.accuracy(out, gt)

        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_epoch=True)
        
        # self.logger.experiment.add_scalar("Loss/Train", loss, self.current_epoch)
        # self.logger.experiment.add_scalar("Accuracy/Train", acc, self.current_epoch)

        return loss
    
    def validation_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        self.log("val_loss", loss, prog_bar=True, on_epoch=True)

        acc = self.accuracy(out, gt)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        
        # self.logger.experiment.add_scalar("Loss/Valid", loss, self.current_epoch)
        # self.logger.experiment.add_scalar("Accuracy/Valid", acc, self.current_epoch)

        return loss
    
    def test_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        acc = self.accuracy(out, gt)
        loss = self.criterion(out, gt)
        self.log('test_acc', acc, on_step=False, on_epoch=True)
        self.log("test/loss", loss, prog_bar=True, on_epoch=True)
        return {"loss": loss, "outputs": out, "gt": gt}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
dm = StanfordCarsDataModule(batch_size=32)

model = LitModel((3, 224, 224), 196, transfer=True, learning_rate=0.0001)

trainer = pl.Trainer(max_epochs=50, accelerator="gpu", logger=WandbLogger(project="Reduction"))

In [ ]:
trainer.fit(model, dm)

In [ ]:
trainer.test(model, dm)